# Mini-Project 1: Galaxy Clusters & Velocity

**Context**

This notebook illustrates the basics of accessing a galaxy catalog from N-body simulations through Apache Spark as well as how to select useful samples of data to study galaxy clusters. Data would be representative (although we will play with a very small set) to LSST simulations data. For more about Apache Spark in the context of astronomy, connect to [AstroLab Software](https://astrolabsoftware.github.io/)!

**Learning objectives**

After going through this notebook, you should be able to:

- Load and efficiently access a galaxy catalog with Apache Spark
- Apply cuts to the catalog using Spark SQL functionalities
- Have several example of quality cuts and validation procedures 
- Derive scientific results on galaxy clusters
- Distribute the computation and the plotting routine to be faster!

In [ ]:
# this is what you would probably need
from typing import Iterator, Generator, Any

import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt 

from pyspark.sql.functions import pandas_udf , PandasUDFType
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

## Loading data with Spark

Let's load the `data/catalogs` data set:

In [ ]:
# Path to the data
fn = "../data/catalogs"

# Load the data - this a lazy operation, no data movement yet!
df = spark.read.format("parquet").load(fn)

# Let's inspect the schema
df.printSchema()

# Number of objects in the catalog
print("Number of rows: {}".format(df.count()))

Let's have a look a some mass values. Apache Spark provides filter mechanisms, which you can use to speed up data access if you only need a certain chunks of the dataset:

In [ ]:
# We reject synthetic halos (negative halo_id) that are added to host the ultra-faint galaxies
cols = ["halo_mass", "stellar_mass", "blackHoleMass", "halo_id"]
df.filter("halo_id > 0").select(cols).show(5)

Note that `halo_mass` is duplicated for all the members of the same halo. 
We can also easily look at statistics about individual columns:

In [ ]:
# Let's look at the stellar_mass and halo_mass distributions
df.select(["stellar_mass", "halo_mass", "redshift"]).describe().show(5)

## Halo mass distribution

To start this journey, let's look at the distribution of halo masses in the catalog.

**Exercise (£):** Create 3 DataFrames from `df` with different populations:
- Full data set (all redshift)
- low-z (0.0 < z < 0.2) clusters
- high-z (2.5 < z < 3.1) clusters

In the three cases, you will select only central galaxies, and clusters with positive `halo_id` (i.e. we reject the synthetic halos that are added to host the ultra-faint galaxies). (hint: to speed up the computation, do not forget cache capabilities!)

Since the number of clusters is rather high, we will capitalize on the fact that we are doing computation in parallel.
The way to be faster is to distribute the computation which leads to the data to be plotted. Histograms are particularly easy to distribute:
- Load the data set (distributed accross machines)
- Apply filters on lines and select columns (order does not matter as Spark will choose the optimal way). Partitions will be processed in parallel. If you have more partitions than workers (typically CPU), there will be a partition queue.
- With the remaining data in each partition, build an histogram per partition.
- Reduce to the driver all partition histograms by summing them up. You have the final histogram!

**Exercise (£££):** Write such a method to be applied on each Spark partition to compute histograms in parallel (each would contain only a fraction of the data). Hint: `mapPartitions` and `numpy` could be your friends.

### A weird feature!

We selected the clusters based on their `halo_id`, and to avoid double counting we took only entries corresponding to central galaxies (all galaxies for a given `halo_id` have the same `halo_mass`).

**Exercise (£):** Inspect the `halo_mass` distribution values. What do you observe? Hint: did you see the attack of the clones? (and also capitalise on the histogram method)

## Galaxy clusters and velocity in DC2

### A few individual galaxy clusters

Let's now focus on some selected galaxy clusters. 

**Exercise (£):** Extract 5 rich clusters ($n_{gal} > 20$).

### Mean velocity as a function of redshift

To end this journey, let's have a look at the mean velocity distribution as a function of redshift:

In [ ]:
# Redshift range
redshift_start = 0.0
redshift_stop = 3.0
redshift_step = 0.5
redshift_window = 0.1
values = np.arange(redshift_start, redshift_stop, redshift_step)

# start at 0.2 because stat is poor at very low redshift
values[0] = 0.2

**Exercise (££):** Make histograms of the mean 3D velocity of haloes, at different redshift ranges. Hint: look at `groupBy` and `agg`.

## The velocity dispersion–halo mass relation

**Exercise (££££):** compute the velocity dispersion–halo mass relation. The principle is similar to the M-$\sigma$ relation for stars around black holes. The idea is to highlight the fact that gravitational interaction/friction between galaxies has an effect for the cluster galaxy evolution.

Hints:
- Use pandas UDF to compute the 3D velocity norm.
- Use `stddev_pop` combined to `groupBy`.
- Use DataFrame `join` method.

### To go further: fitting the data

Following e.g. [1602.00611](https://arxiv.org/abs/1602.00611) (see section 4.1), we can model the mean relation between velocity dispersion and halo mass at a given redshift using a simple power-law of the form:

$$\sigma_v(\rm{M}_h) = \alpha \Big( \dfrac{\rm{M}_h}{10^{14}\rm{M}_\odot} \Big)^{\beta}$$

and the dependency in redshift is given by

$$\sigma_v(z) = \sigma_v(0)\sqrt{1 + z}$$